## Initialisation

In [ ]:
from __future__ import annotations

from pathlib import Path

import mqt.pathfinder.utils as utils
from mqt.pathfinder import cost_functions as cf
from mqt.pathfinder.graph import Graph

In [ ]:
with Path("graph").open() as file:
    graph = Graph.read(file)
graph.plot()

## QUBO Construction

### Parameters

In [ ]:
encoding_type = cf.EncodingType.ONE_HOT
n_paths = 1
max_path_length = graph.n_vertices

In [ ]:
settings = cf.PathFindingQUBOGeneratorSettings(encoding_type, n_paths, max_path_length)
generator = cf.PathFindingQUBOGenerator(cf.MinimisePathLength([1], loop=True), graph, settings)

### Constraints

In [ ]:
generator.add_constraint(cf.PathIsValid([1]))
generator.add_constraint(cf.PathIsLoop([1]))
generator.add_constraint(cf.PathContainsVerticesExactlyOnce(graph.all_vertices, [1]))

### Generate QUBO Formulation

In [ ]:
generator.construct()

In [ ]:
generator.construct_expansion()

In [ ]:
A = generator.construct_qubo_matrix()
utils.print_matrix(A)

## Test Results

### Brute Force Optimisation

In [ ]:
import numpy as np

(best_test, best_score) = utils.optimise_classically(A)

x = np.array(best_test)
pth = generator.decode_bit_array(best_test)
print(pth)

### Operator: Classical Eigensolver

In [ ]:
from typing import TYPE_CHECKING

import numpy as np
import numpy.typing as npt
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit.result import QuasiDistribution

if TYPE_CHECKING:
    from qiskit.quantum_info import Statevector


def bitfield(n: int, L: int) -> list[int]:
    result = np.binary_repr(n, L)
    return [int(digit) for digit in result]


def sample_most_likely(state_vector: QuasiDistribution | Statevector) -> npt.NDArray[np.int_ | np.float64]:
    """Compute the most likely binary string from state vector.
    Args:
        state_vector: State vector or quasi-distribution.

    Returns:
        Binary string as an array of ints.
    """
    values = list(state_vector.values()) if isinstance(state_vector, QuasiDistribution) else state_vector
    n = int(np.log2(len(values)))
    k = np.argmax(np.abs(values))
    x = bitfield(k, n)
    x.reverse()
    return np.asarray(x)


op = generator.construct_operator()

npme = NumPyMinimumEigensolver()
result = npme.compute_minimum_eigenvalue(op)
x = sample_most_likely(result.eigenstate)
print(generator.decode_bit_array(x))
print(result.eigenvalue)

### QAOA

In [ ]:
import random

from qiskit.algorithms.minimum_eigensolvers import QAOA
from qiskit.algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler
from qiskit.utils import algorithm_globals

for _ in range(100):
    seed = random.randint(10000, 20000)
    algorithm_globals.random_seed = seed
    op = generator.construct_operator()

    sampler = Sampler()
    optimizer = COBYLA()
    qaoa = QAOA(sampler, optimizer, reps=5)

    result = qaoa.compute_minimum_eigenvalue(op)
    x = sample_most_likely(result.eigenstate)
    print(seed)
    print(generator.decode_bit_array(x))
    print(result.eigenvalue)